In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [222]:
import re

In [526]:
import io
import zipfile
import pandas as pd
import requests

TODO

Mandatory
- decide how many years to allow
- raise value error if geo_unit entry is not acceptable
- select the column where table-specific data begins. It's the thumber
    - sequence_table_df['start_position'].loc[sequence_table_df['table_id'] == table].max()
    - then select columns [0:6], plus the table columns

Optional
- make a function to access the zipped & nonzipped files? Becuase need to read in as request, then pass to zipfile or pandas.
- make a function to clean up column names

### Get and format imputs.

In [538]:
# The 5 user inputs will be

year = 2015
state_abbr = 'AL' 
table = 'b00002'
geo_unit = 'non_block' # data for block level only or all other geographies. Alternate: 'block'
content_type = 'estimate' # retrieve estimates or margins of error. Alternate: 'margin' 

In [539]:
# First run the `state_dict` cell below. 

# Tranform/format arguments

year = str(year) # todo: confirm this is needed

# standardize formatting
state_abbr_upper = state_abbr.upper() # uppercase used in geographic call later
state_abbr = state_abbr.lower()

table = table.upper()

# Get geographic level of data to use in data import url

# **Laurel** I don't think I've seen code formatted like the if statments below. Usually they're inside of another function. Is this ok?

if geo_unit == 'block':
    url_geo_unit = 'Tracts_Block_Groups_Only'
if geo_unit == 'non_block':
    url_geo_unit = 'All_Geographies_Not_Tracts_Block_Groups'


# Get full state name. 
state_name = state_dict[state_abbr]

# Get sequence number
sequence_table_url = 'http://www2.census.gov/programs-surveys/acs/summary_file/{0}/documentation/user_tools/ACS_5yr_Seq_Table_Number_Lookup.xls'.format(year)
sequence_table_df = pd.read_excel(sequence_table_url, dtype=str)

# clean up column names
sequence_table_df.columns = sequence_table_df.columns.str.replace(' ', '_').str.lower()

# find the sequence number of the first occurance of the desired table and left pad it
sequence = sequence_table_df['sequence_number'].loc[sequence_table_df['table_id'] == table].max()

padded_sequence = '{:0>4}'.format(sequence_table_df['sequence_number'].loc[sequence_table_df['table_id'] == table].max())


In [577]:
sequence

'1'

In [584]:
# Get the number of columns at the beginning of the dataframe that are common to all tables in the df.
# ** Laurel **
    # Why does .min() work here rather than .max()? See df head below.

# The first table column, so core columns are that number minus 2

# Harcoding for now b/c last common column always seems to be 6
last_common_column = 5 
#last_common_column = int(sequence_table_df['start_position'].loc[sequence_table_df['table_id'] == table].min()) - 2

In [579]:
last_common_column

7

### Import raw files

In [541]:
# raw data file
# https://stackoverflow.com/questions/9419162/download-returned-zip-file-from-url/14260592

#working: 'https://www2.census.gov/programs-surveys/acs/summary_file/2015/data/5_year_seq_by_state/Alabama/All_Geographies_Not_Tracts_Block_Groups/20155al0002000.zip'
url = 'https://www2.census.gov/programs-surveys/acs/summary_file/{0}/data/5_year_seq_by_state/{1}/{2}/{0}5{3}{4}000.zip'.\
format(year, state_name, url_geo_unit, state_abbr,padded_sequence)

results = requests.get(url)
# raise status?

zipped = zipfile.ZipFile(io.BytesIO(results.content))

for value in content_type:
    if content_type == 'margin':
        raw_csv = zipped.extract('m{0}5{1}{2}000.txt'.format(year,state_abbr, padded_sequence)) # saves in pwd, unless other is specified
        raw_df = pd.read_csv(raw_csv, dtype=str,header=None)
    elif content_type == 'estimate':
        raw_csv = zipped.extract('e{0}5{1}{2}000.txt'.format(year, state_abbr, padded_sequence))
        raw_df = pd.read_csv(raw_csv, dtype=str,header=None)
    else:
        print('no file') # need this?

zipped.close()

In [542]:
# Header file


In [543]:
# Import header file

    # 1) download zip file here, e.g.: #https://www2.census.gov/programs-surveys/acs/summary_file/2015/data/2015_5yr_Summary_FileTemplates.zip?#

    # 2) extract the file with format 'Seq{0}.xls'.format(sequence|)
        # select estimate or margin of error tab
    # url above and unpadded number for this file extraction.

In [544]:
# ** Laurel **
# Should I not reuse the same variables?

url = 'https://www2.census.gov/programs-surveys/acs/summary_file/{0}/data/{0}_5yr_Summary_FileTemplates.zip'.format(year)
results = requests.get(url)

zipped = zipfile.ZipFile(io.BytesIO(results.content))
header_excel = zipped.extract('Seq{0}.xls'.format(sequence))

header_df = pd.read_excel(header_excel, dtype=str)

zipped.close()

In [545]:
header_df.columns

Index(['FILEID', 'FILETYPE', 'STUSAB', 'CHARITER', 'SEQUENCE', 'LOGRECNO',
       'B00001_001', 'B00002_001'],
      dtype='object')

In [547]:
%%time
#https://www2.census.gov/programs-surveys/acs/summary_file/2015/documentation/geography/5_year_Mini_Geo.xlsx
# Runtime issue - around 1 min
# I could keep this in the final repo if it would help
geo_df = pd.read_excel('https://www2.census.gov/programs-surveys/acs/summary_file/{0}/documentation/geography/5_year_Mini_Geo.xlsx'.format(year),
              sheet_name = state_abbr_upper, 
              dtype=str)

CPU times: user 41.6 s, sys: 514 ms, total: 42.1 s
Wall time: 47.4 s


In [548]:
geo_df.sample(5)

,STATE,LOGRECNO,GEOID,Name
5370,AL,0005371,15000US010970037103,"Block Group 3, Census Tract 37.10, Mobile Coun..."
2868,AL,0002869,14000US01125010303,"Census Tract 103.03, Tuscaloosa County, Alabama"
3829,AL,0003830,15000US010550003003,"Block Group 3, Census Tract 3, Etowah County, ..."
5339,AL,0005340,15000US010970034082,"Block Group 2, Census Tract 34.08, Mobile Coun..."
7065,AL,0007066,16000US0107672,"Blue Springs town, Alabama"


### Append raw data and header files.

In [551]:
# Change raw_df column names to align with header_df column names
raw_df.columns = header_df.columns

In [552]:
header_df.head()

,FILEID,FILETYPE,STUSAB,CHARITER,SEQUENCE,LOGRECNO,B00001_001,B00002_001
0,FILEID,FILETYPE,STUSAB,CHARITER,SEQUENCE,LOGRECNO,UNWEIGHTED SAMPLE COUNT OF THE POPULATION for ...,UNWEIGHTED SAMPLE HOUSING UNITS for Universe: ...


In [583]:
raw_df.head(2)

,FILEID,FILETYPE,STUSAB,CHARITER,SEQUENCE,LOGRECNO,B00001_001,B00002_001
0,ACSSF,2015e5,al,000,0001,0000001,408374,186794
1,ACSSF,2015e5,al,000,0001,0000002,227181,101113


In [554]:
joined_df = header_df.append(raw_df)

In [555]:
# Reindex becasue line above results in 2 rows with index 0
joined_df = joined_df.reset_index(drop=True)

### Select the requested table's columns

In [556]:
# select common columns + table columns
# ** Laurel ** : best to do in two steps for clarity or one for brevity?
# replace column names with first row.
# delete table and universe text from column names. 

In [580]:
last_common_column

7

In [581]:
# Select the columns that all tables in the df have in common
common_df = joined_df.iloc[:, 0: last_common_column]

In [558]:
# select the columns for the table to be returned
table_df = joined_df.loc[:, joined_df.columns.str.startswith('{0}_'.format(table))]

In [582]:
common_df.head()

,FILEID,FILETYPE,STUSAB,CHARITER,SEQUENCE,LOGRECNO,B00001_001
0,FILEID,FILETYPE,STUSAB,CHARITER,SEQUENCE,LOGRECNO,UNWEIGHTED SAMPLE COUNT OF THE POPULATION for ...
1,ACSSF,2015e5,al,000,0001,0000001,408374
2,ACSSF,2015e5,al,000,0001,0000002,227181
3,ACSSF,2015e5,al,000,0001,0000003,181193
4,ACSSF,2015e5,al,000,0001,0000004,347141


In [559]:
# Final data columns
df = pd.concat([common_df, table_df],axis=1)

In [560]:
#df.head()

In [561]:
# replace column names with first row.
df.columns = df.iloc[0]
df.drop([0], inplace = True)

In [562]:
# Clean up table column names by removing everything before the `sequence_table_df` universe field
# todo: whats up with the %s?

In [563]:
# regex pattern 
    # any character to the start of the line. 
# what to do with it 
universe = (sequence_table_df['table_title'].loc[(sequence_table_df['table_id'] == table) & \
                                        (sequence_table_df['start_position'] == 'nan') & \
                                        (sequence_table_df['line_number'] == 'nan')]).to_string(index=False)
# line, start are null? second record? 
#(sequence_table_df.line_number.isna) & (sequence_table_df.start_position.isna) & 
#sequence_table_df['sequence_number'].loc[sequence_table_df['table_id'] == table].max()

In [564]:
df.columns = df.columns.str.replace(r'^.*{0}'.format(universe),'', regex=True)

### Merge geographic location file.
Merge on LOGRECNO
Optional: drop state column

In [565]:
df = df.merge(geo_df, on='LOGRECNO', how='left')


In [567]:
df.head()

,FILEID,FILETYPE,STUSAB,CHARITER,SEQUENCE,LOGRECNO,UNWEIGHTED SAMPLE COUNT OF THE POPULATION for Universe: Total Population% Total,% Total,STATE,GEOID,Name
0,ACSSF,2015e5,al,000,0001,0000001,408374,186794,AL,04000US01,Alabama
1,ACSSF,2015e5,al,000,0001,0000002,227181,101113,AL,04001US01,Alabama -- Urban
2,ACSSF,2015e5,al,000,0001,0000003,181193,85681,AL,04043US01,Alabama -- Rural
3,ACSSF,2015e5,al,000,0001,0000004,347141,156521,AL,040A0US01,Alabama -- In metropolitan or micropolitan sta...
4,ACSSF,2015e5,al,000,0001,0000005,291673,130477,AL,040C0US01,Alabama -- In metropolitan statistical area


In [499]:
# move the geographic columns back to the end of the common columns. 3 columns
# move -1:-3 ?) and insert after at last common column - 1
# last_common_column = 6 


In [500]:
geo_cols = [df.iloc[:,-1], df.iloc[:,-2]]

In [501]:
df.drop(df.columns[[-1,-2,-3]], axis=1, inplace=True)

In [502]:
df.head()

,FILEID,FILETYPE,STUSAB,CHARITER,SEQUENCE,LOGRECNO,% Total
0,ACSSF,2015e5,al,000,0001,0000001,408374
1,ACSSF,2015e5,al,000,0001,0000002,227181
2,ACSSF,2015e5,al,000,0001,0000003,181193
3,ACSSF,2015e5,al,000,0001,0000004,347141
4,ACSSF,2015e5,al,000,0001,0000005,291673


In [503]:
# for each element in the list of geo column series, insert it into postion 6, or last common column
for i in geo_cols:
    df.insert(last_common_column, i.name, i, allow_duplicates=True)

In [504]:
df.head()

,FILEID,FILETYPE,STUSAB,CHARITER,SEQUENCE,LOGRECNO,GEOID,Name,% Total
0,ACSSF,2015e5,al,000,0001,0000001,04000US01,Alabama,408374
1,ACSSF,2015e5,al,000,0001,0000002,04001US01,Alabama -- Urban,227181
2,ACSSF,2015e5,al,000,0001,0000003,04043US01,Alabama -- Rural,181193
3,ACSSF,2015e5,al,000,0001,0000004,040A0US01,Alabama -- In metropolitan or micropolitan sta...,347141
4,ACSSF,2015e5,al,000,0001,0000005,040C0US01,Alabama -- In metropolitan statistical area,291673


In [ ]:
# Clean columns

In [511]:
df.columns.str.replace(' ','_')\
          .str.replace('%' , 'percent')\
          .str.replace('(' , '_')\
          .str.replace (')' , '_')\
          .str.lower()

Index(['fileid', 'filetype', 'stusab', 'chariter', 'sequence', 'logrecno',
       'geoid', 'name', 'percent_total'],
      dtype='object')

In [515]:
table_id +'_'+state_name = df.copy()

SyntaxError: can't assign to operator (<ipython-input-515-a861c5a8a6df>, line 1)

In [517]:
new_name = table +'_'+state_name

AttributeError: module 'pandas' has no attribute 'name'

In [276]:
state_dict = {
'ak':'alaska',
'al':'Alabama',
'ar':'arkansas',
'az':'arizona',
'ca':'california',
'co':'colorado',
'ct':'connecticut',
'dc':'districtofcolumbia',
'de':'delaware',
'fl':'florida',
'ga':'georgia',
'gu':'guam',
'hi':'hawaii',
'ia':'iowa',
'id':'idaho',
'il':'illinois',
'in':'indiana',
'ks':'kansas',
'ky':'kentucky',
'la':'louisiana',
'ma':'massachusetts',
'md':'maryland',
'me':'maine',
'mi':'michigan',
'mn':'minnesota',
'mo':'missouri',
'ms':'mississippi',
'mt':'montana',
'na':'national',
'nc':'northcarolina',
'nd':'northdakota',
'ne':'nebraska',
'nh':'newhampshire',
'nj':'newjersey',
'nm':'newmexico',
'nv':'nevada',
'ny':'newyork',
'oh':'ohio',
'ok':'oklahoma',
'or':'oregon',
'pa':'pennsylvania',
'pr':'puertorico',
'ri':'rhodeisland',
'sc':'southcarolina',
'sd':'southdakota',
'tn':'tennessee',
'tx':'texas',
'ut':'utah',
'va':'virginia',
'vi':'virginislands',
'vt':'vermont',
'wa':'washington',
'wi':'wisconsin',
'wv':'westvirginia',
'wy':'wyoming' 
}